[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KoltonHauck/Agents_and_GraphRAG/blob/main/Agents%26GraphRAG.ipynb)

[Other Notebook](https://github.com/KoltonHauck/BMI6016_VectorDB/blob/main/BMI6016-VectorDB.ipynb)

# Installing Dependencies

In [ ]:
pip install openai numpy pandas scikit-learn sentence-transformers rank-bm25 faiss-cpu neo4j

# Imports

In [ ]:
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import faiss
from openai import OpenAI
import neo4j

# Embedding Methods

Convert text (or anything) into a vector representation.


In [ ]:
# Sample corpus
documents = [
    "The patient presents with acute chest pain and shortness of breath. An ECG was performed, showing ST elevation in leads II, III, and aVF.",
    "The patient experienced minor discomfort in the chest, but ECG results were normal, ruling out myocardial infarction.",
    "Aspirin and nitroglycerin were administered to manage angina symptoms before transfer to the cardiac unit.",
    "The cardiologist performed an angioplasty with stent placement in the right coronary artery.",
    "The patient complained of chest tightness and a persistent cough, with imaging confirming pneumonia.",
    "Diabetes mellitus with uncontrolled hyperglycemia required insulin therapy adjustment."
]

# BM25 Embeddings
tokenized_corpus = [doc.split(" ") for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)
bm25_scores = {doc: bm25.get_scores(doc.split(" ")) for doc in documents}

# TF-IDF Embeddings
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents).toarray()
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

def get_tfidf_vector(doc):
    return dict(zip(tfidf_feature_names, tfidf_vectorizer.transform([doc]).toarray()[0]))

tfidf_vectors = {doc: get_tfidf_vector(doc) for doc in documents}

# Sentence Transformer Embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(documents, convert_to_numpy=True)
embeddingsDict = {documents[i]: embedding for i,embedding in enumerate(embeddings)}

test_doc = documents[0]
print(f"""Document: '{test_doc}'
                BM25 embedding (len: {len(bm25_scores[test_doc])}): {bm25_scores[test_doc]}
              TF-IDF embedding (len: {len(tfidf_vectors[test_doc])}): {tfidf_vectors[test_doc]}
Sentence Transformer embedding (len: {len(embeddingsDict[test_doc])}): {embeddingsDict[test_doc]}
""")


In [ ]:
for doc, emb in bm25_scores.items():
  print(doc, emb)

In [ ]:
def apply_pca(embedding_dict, n_components=2):
    """Applies PCA to reduce the dimensionality of embeddings."""
    doc_keys = list(embedding_dict.keys())
    vectors = np.array(list(embedding_dict.values()))

    pca = PCA(n_components=n_components)
    reduced_vectors = pca.fit_transform(vectors)

    return {doc: reduced_vectors[i] for i, doc in enumerate(doc_keys)}

# Apply PCA to each embedding method
bm25_pca = apply_pca(bm25_scores)
tfidf_pca = apply_pca({k: list(v.values()) for k,v in tfidf_vectors.items()})
sentence_pca = apply_pca(embeddingsDict)

# Print PCA reduced embeddings
print("BM25 PCA Reduced:", bm25_pca)
print("TF-IDF PCA Reduced:", tfidf_pca)
print("Sentence Transformer PCA Reduced:", sentence_pca)

# Plot PCA results
plt.figure(figsize=(8, 6))

# Extract coordinates
def extract_coordinates(pca_dict):
    return np.array(list(pca_dict.values()))

bm25_coords = extract_coordinates(bm25_pca)
tfidf_coords = extract_coordinates(tfidf_pca)
sentence_coords = extract_coordinates(sentence_pca)

plt.scatter(bm25_coords[:, 0], bm25_coords[:, 1], color='red', label='BM25')
plt.scatter(tfidf_coords[:, 0], tfidf_coords[:, 1], color='blue', label='TF-IDF')
plt.scatter(sentence_coords[:, 0], sentence_coords[:, 1], color='green', label='Sentence Transformer')

# Annotate points
for doc, coord in enumerate(bm25_pca.values()):
    plt.annotate(doc, (coord[0], coord[1]), fontsize=9, color='red')
for doc, coord in enumerate(tfidf_pca.values()):
    plt.annotate(doc, (coord[0], coord[1]), fontsize=9, color='blue')
for doc, coord in enumerate(sentence_pca.values()):
    plt.annotate(doc, (coord[0], coord[1]), fontsize=9, color='green')

plt.title("PCA Projection of Embeddings")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend()
plt.show()


# Search

There are different ways to compare two vectors:
![](https://pbs.twimg.com/media/GDTlkNqWsAA65BZ.jpg)


Most common you'll probably see are euclidean and cosine similarity.

Euclidean: the distance between the two 'points' / vector
Cosine Similarity: the angle between the two 'points' / vector



In [ ]:
doc2cosine_sim = {}
doc2euclidean_dist = {}

for doc_i in documents:
  doc2cosine_sim[doc_i] = {}
  doc2euclidean_dist[doc_i] = {}
  for doc_j in documents:
    if doc_j == doc_i:
      continue
    doc2cosine_sim[doc_i][doc_j] = cosine_similarity([embeddingsDict[doc_i]], [embeddingsDict[doc_j]])[0][0]
    doc2euclidean_dist[doc_i][doc_j] = euclidean_distances([embeddingsDict[doc_i]], [embeddingsDict[doc_j]])[0][0]


for doc, score_results in doc2cosine_sim.items():
  print(doc)
  for doc_j, score in score_results.items():
    print(score, doc_j)
  print()

In [ ]:
# Create a FAISS index
dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension) # Euclidean
index = faiss.IndexFlatIP(dimension) # IP (Inner Product) - Cosine Similarity
index.add(embeddings)

# Sample Queries
query = "What did the patient present with?"
query = "What were the presenting symptoms?"
query = "What medications were given?"

query_embedding = model.encode([query], convert_to_numpy=True)

# Search for nearest neighbors
distances, indices = index.search(query_embedding, k=2)

# Print results
print("Query:", query)
for i, idx in enumerate(indices[0]):
    print(f"Match {i+1}: {documents[idx]} (Distance: {distances[0][i]:.4f})")


# RAG

Here we are using OpenAI to test for a few reasons:
- great model
- easy to use SDK (and very well documented)
- don't have much hardware to run local open source models

(Downloading and running Open source models is a conversation for another time)

In [ ]:
client = OpenAI(
    api_key=""
)

# Function to retrieve relevant context
def retrieve_context(query, k=2):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k=k)
    return [documents[i] for i in indices[0]]

# Function to generate response using OpenAI
def generate_response(query, client):
    context = retrieve_context(query)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Example query
query = "What did the patient present with?"
response = generate_response(query, client)
print("Query:", query)
print("Response:", response)


# Knowledge Graph

![Knowledge Graph](https://valkyrie.ai/wp-content/uploads/2024/04/knowledgeGraphs_-1350x675-1.jpg)

## Some Basic Steps for Constructing / Using
1. Entity and Relationship Extraction (constructing)
2. Ingestion into Graph (constructing)
3. Graph Enrichment (constructing)

4. Graph Querying (using)
5. Agentic Graph (using)

## Neo4j Information

[Neo4j Documentation](https://neo4j.com/docs/cypher-manual/current/introduction/)

[Indexes](https://neo4j.com/docs/cypher-manual/current/indexes/)

[Full Text Index](https://neo4j.com/docs/cypher-manual/current/indexes/semantic-indexes/full-text-indexes/)

[Neo4j Neo4j Python SDK](https://neo4j.com/docs/python-manual/current/)

[Neo4j GenAI Python SDK](https://neo4j.com/docs/neo4j-graphrag-python/current/)

[OpenAI Python SDK](https://github.com/openai/openai-python)

In [ ]:
from neo4j import GraphDatabase
import json
from openai import OpenAI

client = OpenAI(
    api_key=""
)

In [15]:

# Neo4j connection details
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "password") # NEED TO CHANGE

def create_graph(driver):
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")  # Clear existing data

        entities = [
            {"name": "Retrieval-Augmented Generation", "type": "Concept", "description": "A method that enhances LLM responses with document retrieval."},
            {"name": "FAISS", "type": "Tool", "description": "A library for efficient similarity search of embeddings."},
            {"name": "Sentence Transformers", "type": "Model", "description": "A model that converts text into dense embeddings."},
            {"name": "Neo4j", "type": "Database", "description": "A graph database used for knowledge graphs."}
        ]

        relationships = [
            ("Retrieval-Augmented Generation", "USES", "FAISS"),
            ("Retrieval-Augmented Generation", "USES", "Sentence Transformers"),
            ("Knowledge Graph", "STORES", "Neo4j"),
            ("Neo4j", "SUPPORTS", "Graph Queries")
        ]

        for entity in entities:
            session.run(
                f"CREATE (n:{entity['type'].replace(' ', '_')} {{name: $name, description: $description}})",
                name=entity["name"].replace(" ", "_"),
                description=entity["description"]
            )

        for start, rel, end in relationships:
            session.run(
                "MATCH (a {name: $start}), (b {name: $end}) "
                f"CREATE (a)-[:{rel.replace(' ', '_')}]->(b)",
                start=start.replace(" ", "_"),
                end=end.replace(" ", "_")
            )

        # create fulltext index
        session.run(
            "CREATE FULLTEXT INDEX descriptionFulltextIndex IF NOT EXISTS FOR (n:Concept|Tool|Model|Database) ON EACH [n.description]"
        )

driver = GraphDatabase.driver(URI, auth=AUTH)
create_graph(driver)
print("Graph data ingested successfully.")
driver.close()


Graph data ingested successfully.


In [16]:
# other helpful queries
QUERY_VIEW_INDEXES = "show index"
QUERY_GET_ALL_NODE_TYPES = "call db.labels()"
QUERY_SEE_ALL_FULLTEXT_ANALYZERS = "db.index.fulltext.listAvailableAnalyzers"

# GraphRAG

RAG (but with a Graph as your context source instead of a Vector Index)

In [ ]:
# Create fulltext index
def create_fulltext_index():
    with driver.session() as session:
        session.run("""
        CREATE FULLTEXT INDEX entity_search IF NOT EXISTS
        FOR (n:Entity) ON EACH [n.name, n.description]
        """)

def create_vector_index():
    with driver.session() as session:
        session.run("""
        CREATE INDEX vector_index IF NOT EXISTS
        FOR (n:Entity) ON (n.embedding)
        OPTIONS {indexProvider: 'vector-1.0', indexConfig: {`vector.dimensions`: 384, `vector.similarity_function`: 'cosine'}}
        """)

def retrieve_using_fulltext(query):
    with driver.session() as session:
        result = session.run("""
        CALL db.index.fulltext.queryNodes('descriptionFulltextIndex', $fulltext_query) YIELD node, score
        RETURN node.name, node.description, score
        """, fulltext_query=query)

        return result.data()
    
def retrieve_full_using_fulltext(query):
    with driver.session() as session:
        result = session.run("""
        CALL db.index.fulltext.queryNodes('descriptionFulltextIndex', $fulltext_query)
        YIELD node, score
        MATCH (node)--(related)
        RETURN 
            node.name AS name,
            node.description AS description,
            score,
            collect(
                DISTINCT {
                    name: related.name, 
                    description: related.description
                }
            ) AS relatedNodes
        """, fulltext_query=query)

        return result.data()
    
def retrieve_using_vector(query):
    query_embedding = model.encode(query, convert_to_numpy=True).tolist()
    with driver.session() as session:
        result = session.run("""
        MATCH (n:Entity)
        RETURN n.name, n.description, cosineSimilarity(n.embedding, $query_embedding) AS score
        ORDER BY score DESC LIMIT 2
        """, query_embedding=query_embedding)
        return result.data()

# Setup indexes and ingest data
# create_fulltext_index()
# create_vector_index()

driver = GraphDatabase.driver(URI, auth=AUTH)

# Example query
query = "How does retrieval-augmented generation work?"
fulltext_results = retrieve_using_fulltext(query)
# vector_results = retrieve_using_vector(query)

fulltext_full_results = retrieve_full_using_fulltext(query)

print("Fulltext Search Results:", fulltext_results)
# print("Vector Search Results:", vector_results)

print("Fulltext FULL Search Results:", fulltext_full_results)
driver.close()


In [ ]:
# Function to retrieve relevant context from graph
def retrieve_graph_context(query, fulltext_fn=retrieve_full_using_fulltext):
    fulltext_full_results = fulltext_fn(query)
    return fulltext_full_results

# Function to generate response using OpenAI
def generate_response(query, client, context_fn=retrieve_graph_context):
    context = context_fn(query)
    print(f"CONTEXT: {context}")
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a helpful assistant. You will probably receive context to generate a response. Rely on context to answer the question."},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

driver = GraphDatabase.driver(URI, auth=AUTH)

# Example query
query = "How does retrieval-augmented generation work?"
response = generate_response(query, client)
print("Query:", query)
print("Response:", response)

driver.close()

In [ ]:
# No context response
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "system", "content": "You are a helpful assistant. You will probably receive context to generate a response. Rely on context to answer the question."},
                {"role": "user", "content": query}]
)

print(response.choices[0].message.content)

# Graph Agent

In [13]:
# agent toolbox

def query_neo4j(cypher_query: str, parameters: dict = None) -> list:
    """
    Execute a Cypher query on the Neo4j database.
    Returns a list of records.
    Each record is a dictionary-like object containing key-value pairs of returned fields.
    """
    if parameters is None:
        parameters = {}
    with driver.session() as session:
        result = session.run(cypher_query, **parameters)
        return [record.data() for record in result]
    
def query_index(index_name: str, query: str) -> list:
    cypher_query = (
        f"CALL db.index.fulltext.queryNodes('{index_name}', '{query}') YIELD node, score "
        "RETURN node, score"
    )
    with driver.session() as session:
        result = session.run(cypher_query.format(index_name=index_name, query=query))
        return [record.data() for record in result]

def search_graph(cypher_query: str) -> str:
    """
    The agent can call this function with a Cypher query string.
    We'll pass it to the Neo4j database and return the result as a JSON string.
    """
    results = query_neo4j(cypher_query)
    return json.dumps(results, indent=2)

def search_index(index_name: str, query: str) -> str:
    results = query_index(index_name=index_name, query=query)
    return json.dumps(results, indent=2)


functions = [
    {
        "name": "search_graph",
        "description": "Execute a Cypher query on the Neo4j database and return the results.",
        "parameters": {
            "type": "object",
            "properties": {
                "cypher_query": {
                    "type": "string",
                    "description": "A valid Cypher query to execute."
                }
            },
            "required": ["cypher_query"]
        }
    },
        {
        "name": "search_index",
        "description": "Execute a search query on a Fulltext index in the Neo4j database and return the results.",
        "parameters": {
            "type": "object",
            "properties": {
                "index_name": {
                    "type": "string",
                    "description": "A fulltext index name."
                },
                "query": {
                    "type": "string",
                    "description": "A query to run."
                }
            },
            "required": ["index_name", "query"]
        }
    }
]


In [12]:
# Function to query a 'graph' agent.
# The agent has access to the graph db to answer questions.

def ask_graph_agent(user_question: str) -> str:
    print(f"- NEW QUERY: {user_question}")
    # Step 1: Send the user question + function definitions to the model
    messages = [
        {
            "role": "system",
            "content": (
                "You are a medical record assistant. You have access to a function that can run Cypher queries on "
                "a Neo4j knowledge graph of a synthetic patient record. You should generate relevant queries to "
                "fetch patient information. Then return the final answer in natural language. "
                "# SCHEMA INFORMATION "
                "- All Node Types: ['Diagnosis', 'Sign/Symptom', 'Medication', 'Procedure', 'Measurement', 'Page'] "
                "- All nodes have primary field of 'text' (except 'Page' nodes have 'page_text') "
                "# INDEXES "
                "- 'Diagnosis_FULLTEXT_index' on 'Diagnosis' nodes on 'text' field "
                "- 'Page_FULLTEXT_index' on 'Page' nodes on 'page_text' field"
            )
        },
        {
            "role": "user",
            "content": user_question
        }
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        functions=functions,
        function_call="auto"  # Let the model decide when to call the function
    )
    
    # Step 2: Check if the model wants to call a function
    response_message = response.choices[0].message
    
    if response_message.function_call:
        # The model requested to call a function
        function_name = response_message.function_call.name
        function_args = json.loads(response_message.function_call.arguments)

        print(f"- NEW FN CALL: {function_name} | {function_args}")
        
        if function_name == "search_graph":
            # Step 3: Execute the function
            function_result = search_graph(**function_args)  # pass the cypher_query

            print(f"\t- FN RESULT: {function_result}")
            
            # Step 4: Return that result back to the model to get a final answer
            messages.append(response_message)  # append the function call
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_result
                }
            )
            
            second_response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages
            )
            
            final_answer = second_response.choices[0].message.content
            
            print(f"- FINAL RESP: {final_answer}")

            return final_answer
        elif function_name == "search_index":
            # Step 3: Execute the function
            function_result = search_index(**function_args)  # pass the arguments

            print(f"\t- FN RESULT: {function_result}")
            
            # Step 4: Return that result back to the model to get a final answer
            messages.append(response_message)  # append the function call
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_result
                }
            )
            
            second_response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages
            )
            
            final_answer = second_response.choices[0].message.content
            
            print(f"- FINAL RESP: {final_answer}")

            return final_answer
        else:
            # If there's some other function, handle accordingly
            pass
    
    # Otherwise, if no function call is requested, just return the raw content
    return response_message["content"]


In [ ]:
driver = GraphDatabase.driver(URI, auth=AUTH)

query1 = "What medications were given to the patient?"
query2 = "What diagnoses does the patient have?"
query3 = "What page numbers was documented for levaquin?"

ask_graph_agent(query1)

driver.close()